Importing the required Modules

In [7]:
import random
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.models import Sequential
from keras.optimizers import SGD
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd


In [8]:
lemmatizer = WordNetLemmatizer()

In [6]:
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\prath\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prath\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prath\AppData\Roaming\nltk_data...


True

Init File of Intents.json

In [9]:
words = []
classes = []
documents = []
ignore_words = ["?", "!"]

In [10]:
data_file = open("../intents.json").read()
data_file

'{\n  "intent": [\n    {\n      "tag": "greeting",\n      "patterns": [\n        "Hi",\n        "How are you?",\n        "Hey there, how have you been?",\n        "Is anyone there?",\n        "Hello",\n        "Good day",\n        "Hiii",\n        "Hii"\n      ],\n      "responses": [\n        "Hi there, what can I do for you?",\n        "Hey there, how can I help?"\n      ]\n    },\n    {\n      "tag": "goodbye",\n      "patterns": ["Bye", "See you there", "Goodbye"],\n      "responses": [\n        "See you later, thanks for visiting ",\n        "Have a nice day",\n        "Bye! Come back again soon."\n      ]\n    },\n    {\n      "tag": "thanks",\n      "patterns": ["Thanks", "Thank you", "That\'s helpful", "Nice", "Good"],\n      "responses": ["Happy to help!", "Any time!", "My pleasure"]\n    },\n    {\n      "tag": "about",\n      "patterns": [\n        "about you",\n        "Who are you?",\n        "What are you?",\n        "Who you are?",\n        "what you can do",\n        "W

In [11]:
intents=json.loads(data_file)
intents

{'intent': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you?',
    'Hey there, how have you been?',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Hiii',
    'Hii'],
   'responses': ['Hi there, what can I do for you?',
    'Hey there, how can I help?']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you there', 'Goodbye'],
   'responses': ['See you later, thanks for visiting ',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful", 'Nice', 'Good'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'about',
   'patterns': ['about you',
    'Who are you?',
    'What are you?',
    'Who you are?',
    'what you can do',
    'What is the Purpose of the project'],
   'responses': ['I am AyurBot. I can help determine your prakriti type. Prakriti is our inherent nature, with which we are born. It is the nature of our body in terms of dosha. It dies not change duri

In [22]:
for intent in intents["intent"]:
    for pattern in intent["patterns"]:
        # separate each word from the pattern
        w=nltk.word_tokenize(pattern)
        # add it in the words list
        words.extend(w)
        
        # adding it in the documents with the defined tag.
        documents.append((w,intent["tag"]))

        # adding the classes in the classes list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])



In [26]:
words

['Hi',
 'How',
 'are',
 'you',
 '?',
 'Hey',
 'there',
 ',',
 'how',
 'have',
 'you',
 'been',
 '?',
 'Is',
 'anyone',
 'there',
 '?',
 'Hello',
 'Good',
 'day',
 'Hiii',
 'Hii',
 'Bye',
 'See',
 'you',
 'there',
 'Goodbye',
 'Thanks',
 'Thank',
 'you',
 'That',
 "'s",
 'helpful',
 'Nice',
 'Good',
 'about',
 'you',
 'Who',
 'are',
 'you',
 '?',
 'What',
 'are',
 'you',
 '?',
 'Who',
 'you',
 'are',
 '?',
 'what',
 'you',
 'can',
 'do',
 'What',
 'is',
 'the',
 'Purpose',
 'of',
 'the',
 'project',
 'what',
 'is',
 'your',
 'name',
 'what',
 'should',
 'I',
 'call',
 'you',
 'whats',
 'your',
 'name',
 '?',
 'Could',
 'you',
 'help',
 'me',
 '?',
 'give',
 'me',
 'a',
 'hand',
 'please',
 'Can',
 'you',
 'help',
 '?',
 'What',
 'can',
 'you',
 'do',
 'for',
 'me',
 '?',
 'I',
 'need',
 'a',
 'support',
 'I',
 'need',
 'a',
 'help',
 'support',
 'me',
 'please',
 'Complete',
 'the',
 'work',
 'My',
 'name',
 'user',
 'name',
 'you',
 'are',
 'not',
 'Good',
 'Nothing',
 'You',
 'are',
 

In [20]:
for doc in documents:
    print(doc)

(['Hi'], 'greeting')
(['How', 'are', 'you', '?'], 'greeting')
(['Hey', 'there', ',', 'how', 'have', 'you', 'been', '?'], 'greeting')
(['Is', 'anyone', 'there', '?'], 'greeting')
(['Hello'], 'greeting')
(['Good', 'day'], 'greeting')
(['Hiii'], 'greeting')
(['Hii'], 'greeting')
(['Bye'], 'goodbye')
(['See', 'you', 'there'], 'goodbye')
(['Goodbye'], 'goodbye')
(['Thanks'], 'thanks')
(['Thank', 'you'], 'thanks')
(['That', "'s", 'helpful'], 'thanks')
(['Nice'], 'thanks')
(['Good'], 'thanks')
(['about', 'you'], 'about')
(['Who', 'are', 'you', '?'], 'about')
(['What', 'are', 'you', '?'], 'about')
(['Who', 'you', 'are', '?'], 'about')
(['what', 'you', 'can', 'do'], 'about')
(['What', 'is', 'the', 'Purpose', 'of', 'the', 'project'], 'about')
(['what', 'is', 'your', 'name'], 'name')
(['what', 'should', 'I', 'call', 'you'], 'name')
(['whats', 'your', 'name', '?'], 'name')
(['Could', 'you', 'help', 'me', '?'], 'help')
(['give', 'me', 'a', 'hand', 'please'], 'help')
(['Can', 'you', 'help', '?'], 'h

In [23]:
classes

['greeting',
 'goodbye',
 'thanks',
 'about',
 'name',
 'help',
 'noanswer',
 'prakriti',
 'creators',
 'badwords',
 'acknowledge']

Lemmatizer

In [27]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words

['hi',
 'how',
 'are',
 'you',
 'hey',
 'there',
 ',',
 'how',
 'have',
 'you',
 'been',
 'is',
 'anyone',
 'there',
 'hello',
 'good',
 'day',
 'hiii',
 'hii',
 'bye',
 'see',
 'you',
 'there',
 'goodbye',
 'thanks',
 'thank',
 'you',
 'that',
 "'s",
 'helpful',
 'nice',
 'good',
 'about',
 'you',
 'who',
 'are',
 'you',
 'what',
 'are',
 'you',
 'who',
 'you',
 'are',
 'what',
 'you',
 'can',
 'do',
 'what',
 'is',
 'the',
 'purpose',
 'of',
 'the',
 'project',
 'what',
 'is',
 'your',
 'name',
 'what',
 'should',
 'i',
 'call',
 'you',
 'whats',
 'your',
 'name',
 'could',
 'you',
 'help',
 'me',
 'give',
 'me',
 'a',
 'hand',
 'please',
 'can',
 'you',
 'help',
 'what',
 'can',
 'you',
 'do',
 'for',
 'me',
 'i',
 'need',
 'a',
 'support',
 'i',
 'need',
 'a',
 'help',
 'support',
 'me',
 'please',
 'complete',
 'the',
 'work',
 'my',
 'name',
 'user',
 'name',
 'you',
 'are',
 'not',
 'good',
 'nothing',
 'you',
 'are',
 'bad',
 'noting',
 'to',
 'worry',
 'chatgpt',
 'is',
 'best

In [29]:
words = sorted(list(set(words)))
words

["'re",
 "'s",
 ',',
 'a',
 'about',
 'annoying',
 'anyone',
 'are',
 'away',
 'bad',
 'be',
 'been',
 'best',
 'by',
 'bye',
 'ca',
 'call',
 'can',
 'chatgpt',
 'complete',
 'could',
 'created',
 'creator',
 'day',
 'despise',
 'determine',
 'do',
 'figure',
 'find',
 'for',
 'fuck',
 'give',
 'go',
 'good',
 'goodbye',
 'hand',
 'hate',
 'have',
 'hello',
 'help',
 'helpful',
 'hey',
 'hi',
 'hii',
 'hiii',
 'how',
 'i',
 'identify',
 'irrelevant',
 'is',
 'made',
 'make',
 'me',
 'my',
 "n't",
 'name',
 'need',
 'nice',
 'not',
 'nothing',
 'noting',
 'now',
 'of',
 'ok',
 'out',
 'please',
 'prakriti',
 'project',
 'purpose',
 'see',
 'should',
 'shut',
 'sick',
 'so',
 'stand',
 'support',
 'thank',
 'thanks',
 'that',
 'the',
 'there',
 'these',
 'this',
 'to',
 'understand',
 'up',
 'user',
 'what',
 'whats',
 'which',
 'who',
 'work',
 'worry',
 'worthless',
 'yeah',
 'you',
 'your']

In [30]:
classes = sorted(list(set(classes)))
classes

['about',
 'acknowledge',
 'badwords',
 'creators',
 'goodbye',
 'greeting',
 'help',
 'name',
 'noanswer',
 'prakriti',
 'thanks']

In [31]:
print(len(documents), "documents")

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)

340 documents
11 classes ['about', 'acknowledge', 'badwords', 'creators', 'goodbye', 'greeting', 'help', 'name', 'noanswer', 'prakriti', 'thanks']
97 unique lemmatized words ["'re", "'s", ',', 'a', 'about', 'annoying', 'anyone', 'are', 'away', 'bad', 'be', 'been', 'best', 'by', 'bye', 'ca', 'call', 'can', 'chatgpt', 'complete', 'could', 'created', 'creator', 'day', 'despise', 'determine', 'do', 'figure', 'find', 'for', 'fuck', 'give', 'go', 'good', 'goodbye', 'hand', 'hate', 'have', 'hello', 'help', 'helpful', 'hey', 'hi', 'hii', 'hiii', 'how', 'i', 'identify', 'irrelevant', 'is', 'made', 'make', 'me', 'my', "n't", 'name', 'need', 'nice', 'not', 'nothing', 'noting', 'now', 'of', 'ok', 'out', 'please', 'prakriti', 'project', 'purpose', 'see', 'should', 'shut', 'sick', 'so', 'stand', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'these', 'this', 'to', 'understand', 'up', 'user', 'what', 'whats', 'which', 'who', 'work', 'worry', 'worthless', 'yeah', 'you', 'your']
